**Primera parte: PCA**

En esta primera parte vamos a explorar cómo hacer una PCA usando el paquete ngsTools. ngsTools (Fumagalli et al. 2015) fue diseñado específicamente para trabajar con genotype likelihoods o probabilidad de genotipos. 

La idea detrás del uso de probabilidades de genotipos en vez de genotipos es que normalmente, para tener certeza de que has llamado un genotipo con las herramientas bioinformáticas que tenemos, necesitas una media-buena cobertura (una media de 10-15X). Esto no siempre se consigue, ya que pede haber ADN antiguo, o podemos decidir secuenciar a menor profundidad para hacer el estudio más económico (por ejemplo, 5X en vez de 15X). ngsTools ha demostrado ser una herramienta muy útil capaz de llamar probabilidades de genotipo incluso a 2X. 

La razón por la que en bajas coberturas es mejor usar probabilidades de genotipo que genotipos es porque hay una incertidumbre en cuanto a los SNPs heterocigotos. Por ejemplo, si tenemos 20X y salen 20 A en un sitio, es seguro que es homocigoto. Si salen 9 A y 11 T, es seguro que el sitio es heterocigoto. Pero si tenemos 3X y tenemos 3A o 3T, la probabilidad de perder un alelo ("allelic dropout") es más alta. También es más probable que tengamos falsos alelos, por error de amplificación o de secuenciación.

![Alt text](./maxresdefault.jpg)

Para tu trabajo en PCA, no necesitas usar siempre probabilidades de genotipo. Si tienes un dataset adecado a 20X, puedes usar genotipos tranquilamente. En este caso particular trabajaremos con el cromosoma 38 de varios lobos y perros. Mi dataset tenía genomas a 5X, 7X e incluso 30X, así que para evitar sesgos usaremos simplemente probabilidades de genotipo para todos. 

Los archivos "geno" que tienes fueron generados en ANGSD (un "genotype likelihood caller" o "llamador de probabilidades de genotipo", puedes encontrar su wiki en https://www.popgen.dk/angsd/index.php/ANGSD#Overview). Para hacerlo usé el comando:

In [ ]:
$PROG/angsd/angsd -P 16 -bam $BAM/23genomes.chr01.filelist -ref $REF -out $OUT/chr01.canfam -r chr01 \
	-uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1 \
	-minMapQ 20 -minQ 20 -minInd 23 -setMinDepth 46 -setMaxDepth 742 -doCounts 1 \
	-GL 1 -doMajorMinor 1 -doMaf 1 -skipTriallelic 1 -minIndDepth 5 -sites $SITES -SNP_pval 1e-5 -doGeno 32 -doPost 1

En este paso usé varios parámetros estándares de ANGSD para asegurar la calidad de las probabilidades de genotipo (-uniqueOnly 1 -remove_bads 1 -only_proper_pairs 1 -trim 0 -C 50 -baq 1), filtré sitios con menor calidad de llamado y mapeo que 20 (phred score: 99% seguro), filtré sitios que no estaban presentes en todos los 23 genomas (-minInd 23), puse una profundidad mínima de 2X (46, para 23 muestras), calculé la cobertura acumulada de todos los sitios y filtré sitios 2X por encima de ella para filtrar genes con más de dos copias -por ejemplo, amilasa en perros(-setMaxDepth 742) y llamé genotipos (-doGeno 32 -doPost 1 -GL 1 -doMajorMinor 1 -doMaf 1). 

Tenéis más información sobre estos parámetros en la wiki de ANGSD. De momento, trabajemos en ngsTools. 


In [ ]:
prog=/home/bio/programas
in=/home/bio/inputfiles
out=/directorio/hacia/tu/carpeta/personal 

$prog/ngsTools/ngsPopGen/ngsCovar -probfile $in/chr38.150gen.pca.geno -outfile $out/chr38.150gen.pca.covar -nind 150 -nsites 91243 -call 0 -norm 0

In [3]:
# mafs.gz nos dice cuántos sitios hay:
zcat chr38.150gen.pca.mafs.gz | tail -n+2 | wc -l

#¿cuales son los eigenvectores y eigenvalores?

plink=$prog/plink_linux_x86_64_20240818/plink

$plink --vcf $in/chr38.150gen.pca.vcf --make-bed --out $out/chr38.150gen.pca --dog
$plink --bfile $out/chr38.150gen.pca --pca 20 --dog --out $out/pca_results

gzip: chr38.150gen.pca.mafs.gz: No such file or directory
0


Explora los archivos de eigenvectores e eigenvalores.

In [ ]:
paste $in/pop.info $out/pca_results.eigenvec > $out/pca_with_population.txt

En el ejercicio de ngsTools has creado una matriz de covarianzas.
Vamos a estudiarla: ¿Cuántas columnas tiene? ¿Cuántas filas? 

In [ ]:
wc -l chr38.150gen.pca.covar
cat chr38.150gen.pca.covar | head -1 | awk '{print NF}'

Vamos a crear distintas formas de clusterizar los datos.

Por procedencia de los perros y lobos:

In [ ]:
Rscript -e 'write.table(cbind(seq(1,150),rep(1,150), c(rep("DOG.WE",5), rep("DOG.IT",10), rep("DOG.IB",21), rep("DOG.SC",6), rep("DOG.RU",20), rep("DOG.SIB",6), rep("EEGW",40), rep("SCGW",10), rep("IBGW",32))), row.names=F, sep="\t", col.names=c("FID","IID","CLUSTER"), file="dog.gw.150.cls1.clst", quote=F)'

Por especie:

In [ ]:
Rscript -e 'write.table(cbind(seq(1,150),rep(1,150), c(rep("DOG",68), rep("NEGW", 50), rep("IBGW",32))), row.names=F, sep="\t", col.names=c("FID","IID","CLUSTER"), file="dog.gw.150.cls2.clst", quote=F)'

Ploteemos ahora la PCA. Elige un archivo "clst" y crea tu fichero pdf

In [ ]:
Rscript $prog/ngsTools/Scripts/plotPCA.R -i $out/chr38.150gen.pca.covar -c 1-2 -a $out/dog.gw.150.cls2.clst -o $out/chr38.clst1.canfam.pca.pdf


Puedes descargar el plot en tu ordenador personal usando el comando scp user@172.64.16.4:/ruta/hacia/tus/archivos ./

Preguntas:

**Pregunta 1:** ¿Qué puedes ver en la PCA? ¿Puedes ver la diferencia entre las poblaciones? 

**Pregunta 2:** Prueba a usar otro archivo clst y vuelve a plotearlo. ¿Qué diferencias observas?

**Pregunta 3:** Modifica las PCs : -c 1-2 serían PC1 y PC2. Prueba PC1-PC3, PC2-PC4... tú eliges. ¿Qué patrones observas?